In [7]:
import numpy as np
import pickle
import os
from datetime import datetime, timedelta

class WeatherScaler:
    def __init__(self):
        self.feature_ranges = {
            'temperature': (19, 110),#still in Fahrenheit for model compatibility
            'humidity': (20, 90),
            'wind_speed': (0, 50),
            'precipitation': (0, 20)
        }
    
    def transform(self, data):
        scaled_data = []
        for value, (min_val, max_val) in zip(data, self.feature_ranges.values()):
            scaled_value = (value - min_val) / (max_val - min_val)
            scaled_value = max(0, min(1, scaled_value))
            scaled_data.append(scaled_value)
        return np.array(scaled_data)
    
    def inverse_transform(self, scaled_data):
        unscaled_data = []
        for scaled_value, (min_val, max_val) in zip(scaled_data, self.feature_ranges.values()):
            unscaled_value = scaled_value * (max_val - min_val) + min_val
            if min_val == 0 and max_val == 20:
                unscaled_value = max(0, unscaled_value)
            unscaled_data.append(unscaled_value)
        return np.array(unscaled_data)

def fahrenheit_to_celsius(fahrenheit):
    return (fahrenheit - 32) * 5/9

class WeatherPipeline:
    def __init__(self, sequence_length=1):
        self.sequence_length = sequence_length
        self.num_features = 4
        self.output_days = 5
        model_path = os.path.join(os.getcwd(), 'final4_weather_prediction_model.pkl')
        with open(model_path, 'rb') as f:
            self.model = pickle.load(f)
        self.scaler = WeatherScaler()
    
    def prepare_input(self, input_data):
        if len(input_data) != self.num_features:
            raise ValueError(f"Input data must contain {self.num_features} features")
        scaled_data = self.scaler.transform(input_data)
        model_input = scaled_data.reshape(1, self.sequence_length, self.num_features)
        return model_input
    
    def predict(self, input_data):
        model_input = self.prepare_input(input_data)
        scaled_predictions = self.model.predict(model_input)
        scaled_predictions = scaled_predictions.reshape(self.output_days, self.num_features)
        original_predictions = np.array([
            self.scaler.inverse_transform(day_pred)
            for day_pred in scaled_predictions
        ])
        predictions = []
        feature_names = ['temperature', 'humidity', 'wind_speed', 'precipitation']
        start_date = datetime.now() + timedelta(days=1)
        
        for day in range(self.output_days):
            current_date = start_date + timedelta(days=day)
            original_values = {}
            for i, name in enumerate(feature_names):
                value = float(original_predictions[day][i])
                if name == 'temperature':
                    value = fahrenheit_to_celsius(value)
                original_values[name] = round(value, 2)
            
            day_pred = {
                'date': current_date.strftime('%Y-%m-%d'),
                'day': current_date.strftime('%A'),
                'scaled_values': {
                    name: float(scaled_predictions[day][i])
                    for i, name in enumerate(feature_names)
                },
                'original_values': original_values
            }
            predictions.append(day_pred)
        return predictions

if __name__ == "__main__":
    pipeline = WeatherPipeline()
    current_weather=[40,75,5,3]
    predictions = pipeline.predict(current_weather)
    print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
[{'date': '2024-11-22', 'day': 'Friday', 'scaled_values': {'temperature': 0.6965190172195435, 'humidity': 0.5654170513153076, 'wind_speed': 0.07118898630142212, 'precipitation': -0.004057919606566429}, 'original_values': {'temperature': 27.99, 'humidity': 59.58, 'wind_speed': 3.56, 'precipitation': 0.0}}, {'date': '2024-11-23', 'day': 'Saturday', 'scaled_values': {'temperature': 0.6281790733337402, 'humidity': 0.8295269012451172, 'wind_speed': 0.0025987382978200912, 'precipitation': 0.09739477187395096}, 'original_values': {'temperature': 24.54, 'humidity': 78.07, 'wind_speed': 0.13, 'precipitation': 1.95}}, {'date': '2024-11-24', 'day': 'Sunday', 'scaled_values': {'temperature': 0.6253083944320679, 'humidity': 0.7768123745918274, 'wind_speed': 0.06090688705444336, 'precipitation': -0.03361668810248375}, 'original_values': {'temperature': 24.39, 'humidity': 74.38, 'wind_speed': 3.05, 'precipitation': 0.0}}, {'date': '2024-11-25', 'day': 'Monday', 'sc